In [164]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from pathlib import Path



### 1. Response Selection and Preprocessing

Select **one** of the following response variables:
- Willingness to pay (in euros)
- Number of edits to improve the image

Prepare the dataset by:
- Encoding categorical variables (e.g. creating dummy variables when needed)
- Adding an intercept term

I chose willingness to pay in euros

In [165]:
# Build path to dataset

dataset_path = Path().resolve().parent / "dataset" / "data.csv"

df = pd.read_csv(dataset_path)


In [166]:
# Initial checks

print(f'shape = {df.shape}')
print()
print(df.info())
print()
#print(df.describe(include="all"))

shape = (96, 11)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 11 columns):
 #   Column                                                                                        Non-Null Count  Dtype 
---  ------                                                                                        --------------  ----- 
 0   ID                                                                                            96 non-null     int64 
 1   IMAGE                                                                                         96 non-null     int64 
 2   How scary is this image?                                                                      96 non-null     int64 
 3   How likely are you to see something like this in  a dream?                                    96 non-null     object
 4   What is the dominant emotion of the image?                                                    93 non-null     object
 5   How realistic does t

In [167]:
# Drop some columns

cols_to_drop = [
    "ID",
    "Write a short story inspired by this image without naming any objects in it.",
    "If you were asked to improve this image to make it “perfect”, how many edits would you make?"
]

df = df.drop(columns = cols_to_drop)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 8 columns):
 #   Column                                                         Non-Null Count  Dtype 
---  ------                                                         --------------  ----- 
 0   IMAGE                                                          96 non-null     int64 
 1   How scary is this image?                                       96 non-null     int64 
 2   How likely are you to see something like this in  a dream?     96 non-null     object
 3   What is the dominant emotion of the image?                     93 non-null     object
 4   How realistic does this image look?                            96 non-null     int64 
 5   How likely is this image AI-generated?                         96 non-null     int64 
 6   How much would you pay for this as a painting (in euros)?      96 non-null     object
 7   Does anything feel visually wrong/inconsistent in this image?  96 non-null

In [168]:
# check the column willingness to pay

df_response = df['How much would you pay for this as a painting (in euros)?']

unique_values = set(df_response.dropna())
print(unique_values)

{'100', '18', '25', '1', '50', '0€', '6', '150', '2', '8', '15', '-10', '€5', '0', '10', 'O', '5', '3', '20', 'Zero', '10€'}


In [169]:
# Drop rows containing invalid values

def clean_price(x):
    if pd.isna(x):
        return np.nan

    x = str(x).strip().lower()

    # textual zeros
    if x in ["zero", "o"]:
        return 0.0

    # remove euro symbol
    x = x.replace("€", "")

    # try numeric conversion
    try:
        value = float(x)
    except ValueError:
        return np.nan

    # remove invalid values
    if value < 0:
        return np.nan

    return value

response_col = "How much would you pay for this as a painting (in euros)?"

df[response_col] = df[response_col].apply(clean_price)
df = df.dropna(subset=[response_col])

set(df[response_col].unique())


{np.float64(0.0),
 np.float64(1.0),
 np.float64(2.0),
 np.float64(3.0),
 np.float64(5.0),
 np.float64(6.0),
 np.float64(8.0),
 np.float64(10.0),
 np.float64(15.0),
 np.float64(18.0),
 np.float64(20.0),
 np.float64(25.0),
 np.float64(50.0),
 np.float64(100.0),
 np.float64(150.0)}

In [170]:
# Encoding categorical variables

dream_col = "How likely are you to see something like this in  a dream?"
emotion_col = "What is the dominant emotion of the image?"

df = pd.get_dummies(
    df,
    columns = [dream_col],
    drop_first=True
)

df = pd.get_dummies(
    df,
    columns=[emotion_col],
    drop_first=True
)

In [171]:
wrong_col = "Does anything feel visually wrong/inconsistent in this image?"

df["visually_wrong"] = (
    ~df[wrong_col].str.lower().str.contains("no")
).astype(int)

bool_cols = df.select_dtypes(include="bool").columns
df[bool_cols] = df[bool_cols].astype(int)

df = df.drop(columns=[wrong_col])

print(df.head())

   IMAGE  How scary is this image?  How realistic does this image look?  \
0      1                         6                                    1   
1      1                         3                                    3   
2      1                         8                                    2   
3      1                         1                                    1   
4      1                         1                                    1   

   How likely is this image AI-generated?  \
0                                       8   
1                                      10   
2                                      10   
3                                       2   
4                                       7   

   How much would you pay for this as a painting (in euros)?  \
0                                                2.0           
1                                                0.0           
2                                                8.0           
3                     

In [172]:
# Intercept for the design matrix
response_col = "How much would you pay for this as a painting (in euros)?"

X = df.drop(columns=[response_col])
y = df[response_col].values

X.insert(0, "intercept", 1.0)

bool_cols = X.select_dtypes(include='bool').columns
X[bool_cols] = X[bool_cols].astype(int)
 

X

,intercept,IMAGE,How scary is this image?,How realistic does this image look?,How likely is this image AI-generated?,How likely are you to see something like this in a dream?_Neutral,How likely are you to see something like this in a dream?_Unlikely,How likely are you to see something like this in a dream?_Very unlikely,"What is the dominant emotion of the image?_Anger, confusion",What is the dominant emotion of the image?_Anxiety,...,"What is the dominant emotion of the image?_Sadness, Fear, Anger",What is the dominant emotion of the image?_Stupidity,What is the dominant emotion of the image?_Surprise,What is the dominant emotion of the image?_Tenderness,What is the dominant emotion of the image?_Wonder,What is the dominant emotion of the image?_adventurous,What is the dominant emotion of the image?_feeling of cuteness,What is the dominant emotion of the image?_neutral,What is the dominant emotion of the image?_no emotion / neutral,visually_wrong
0,1.0,1,6,1,8,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1.0,1,3,3,10,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
2,1.0,1,8,2,10,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,1.0,1,1,1,2,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
4,1.0,1,1,1,7,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,1.0,3,1,3,5,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
92,1.0,3,1,2,9,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
93,1.0,3,2,5,4,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
94,1.0,3,3,3,10,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


## Linear Regression Implementation

In [173]:
def fit_ols(X, y):

    XTX = X.T @ X
    XTy = X.T @ y

    beta_hat = np.linalg.pinv(XTX) @ XTy

    return beta_hat

def predict_ols(X, beta):

    return X @ beta

def mse(y_true, y_pred):

    return np.mean((y_true - y_pred) ** 2)

def r2_score(y_true, y_pred):

    ss_res = np.sum((y_true - y_pred) ** 2)
    ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
    return 1 - ss_res / ss_tot




In [ ]:
X_np = X.to_numpy(dtype=float)   # converts all columns to floats
y_np = y.astype(float) 
beta_hat = fit_ols(X.values, y)
pred = predict_ols(X, beta_hat)
y_hat = predict_ols(X.values, beta_hat)

beta_hat = fit_ols(X_np, y_np)
y_hat = predict_ols(X_np, beta_hat)

print("Coefficients:", beta_hat)
print(f'prediction: {pred}')
print("MSE:", mse(y_np, y_hat))
print("R^2:", r2_score(y_np, y_hat))

Coefficients: [-20.01456142   1.41641581   0.97316966   5.62045926  -0.06787859
   2.34253703  13.8701032   -1.42195214  11.40011635  -6.81144798
   4.03182083 -31.09717314 -35.70546891 -17.18843643  11.0607234
   7.74264071  -4.63782622   4.58792872 -23.74070436   1.04018242
  -1.38205494  -5.64774776   7.59416721   2.53272565 -12.06589057
  15.92496621   3.11855725  10.78920903  -0.17213119 -10.77792361
 -20.01016172   2.4145761   -7.01508375  -8.32458588   7.3226007
   5.57602131   9.71218618 -24.63102867   7.76584685   7.14533259
   1.65570041  -8.1920107   -7.56306863   2.70513838]
prediction: 0      1.397799
1      0.405087
2      8.000000
3     -3.114447
4     11.689742
        ...    
91    10.809337
92     1.152874
93    16.621676
94    10.073746
95    10.000000
Length: 95, dtype: float64
MSE: 299.4499611934456
R^2: 0.37212152039809054


## Comparison with Standard Libraries:

In [176]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Create the model
lr = LinearRegression(fit_intercept=False)  # intercept already in X

# Fit the model
lr.fit(X.values, y)  

# Predictions
y_pred_skl = lr.predict(X.values)

# Coefficients
beta_skl = lr.coef_

# MSE and R^2
mse_skl = mean_squared_error(y, y_pred_skl)
r2_skl = r2_score(y, y_pred_skl)

print("scikit-learn Coefficients:", beta_skl)
print("prediction", y_pred_skl)
print("MSE:", mse_skl)
print("R^2:", r2_skl)

scikit-learn Coefficients: [-20.01456142   1.41641581   0.97316966   5.62045926  -0.06787859
   2.34253703  13.8701032   -1.42195214  11.40011635  -6.81144798
   4.03182083 -31.09717314 -35.70546891 -17.18843643  11.0607234
   7.74264071  -4.63782622   4.58792872 -23.74070436   1.04018242
  -1.38205494  -5.64774776   7.59416721   2.53272565 -12.06589057
  15.92496621   3.11855725  10.78920903  -0.17213119 -10.77792361
 -20.01016172   2.4145761   -7.01508375  -8.32458588   7.3226007
   5.57602131   9.71218618 -24.63102867   7.76584685   7.14533259
   1.65570041  -8.1920107   -7.56306863   2.70513838]
prediction [ 1.39779915e+00  4.05086565e-01  8.00000000e+00 -3.11444691e+00
  1.16897420e+01  1.00000000e+01  3.55093231e+00  5.00000000e+00
 -3.18232550e+00  3.10862447e-15 -6.35047570e-14 -8.14121441e+00
 -3.86357613e-14  1.52007361e+01 -6.39512586e+00  4.23431720e+01
 -2.69702229e+00 -4.43246149e-01 -1.99840144e-14 -8.68585687e+00
  5.70090740e+00 -8.21565038e-14 -5.19584376e-14  5.00000

#### The coefficients, MSE, and R² from our scratch implementation of OLS match exactly those obtained from scikit-learn, confirming the correctness of the matrix-based approach. Small numerical differences may occur in larger or ill-conditioned datasets due to different numerical methods used for solving the linear system.

## Interaction Effects and Feature Engineering:

### I picked : How scary is this image? and How realistic does this image look? → numeric, could change the effect of “scary”


In [ ]:
# Assuming X contains your numeric predictors
X_interact = X.copy()

# Interaction term
X_interact['scary_x_realistic'] = X_interact['How scary is this image?'] * X_interact['How realistic does this image look?']

# Convert booleans to int again if needed
bool_cols = X_interact.select_dtypes(include='bool').columns
X_interact[bool_cols] = X_interact[bool_cols].astype(int)

# Convert to NumPy
X_int_np = X_interact.to_numpy(dtype=float)

# Fit
beta_int = fit_ols(X_int_np, y_np)
y_hat_int = predict_ols(X_int_np, beta_int)

# Evaluate
mse_int = mse(y_np, y_hat_int)
r2_int = r2_score(y_np, y_hat_int)

print("MSE with interaction:", mse_int)
print("R² with interaction:", r2_int)

print("MSE without interaction:", mse(y_np, y_hat))
print("R² without interaction:", r2_score(y_np, y_hat))